In [2]:
# 행렬 분해 : R = F * B
# NaN이 포함된 R이 주어졌을 때 행렬 F, B를 추정한다. (by Stochastic Gradient Descent)
import numpy as np
import numba as nb
import matplotlib.pyplot as plt
import pandas as pd

In [193]:
# Load the dataset
df = pd.read_csv("49_Industry_Portfolios_Daily_Value.csv", index_col=0)
df = df.reset_index()

# 1990년 이전 데이터는 보지 않기
df = df[df['index'] > 19900103]

# 결측치가 있는지 검증
df[df.isin([-99.99]).any(axis=1)]

date = pd.to_datetime(df['index'], format='%Y%m%d')
df['index'] = date
df = df.set_index(keys='index')

In [ ]:
# 로그 수익률로 바꾸는 과정

df = df/100
log_returns = np.log(1 + df)
weekly_log_returns = log_returns.resample('W').sum()
weekly_log_returns = weekly_log_returns.reset_index()
weekly_log_returns = weekly_log_returns.drop(['index'], axis=1)
weekly_log_returns.to_csv("49_Industry_Portfolios_Daily_Value_logreturn.csv")

In [175]:
rtn_df = pd.read_csv("49_Industry_Portfolios_Daily_Value_logreturn.csv", index_col=0)
rtn_df

,Agric,Food,Soda,Beer,Smoke,Toys,Fun,Books,Hshld,Clths,...,Boxes,Trans,Whlsl,Rtail,Meals,Banks,Insur,RlEst,Fin,Other
0,0.003383,-0.016165,0.008273,-0.016368,-0.028218,-0.007520,-0.000800,-0.009552,-0.012743,-0.002615,...,-0.006511,-0.010229,-0.011535,-0.005712,-0.026785,-0.006712,-0.015572,-0.001207,-0.008018,-0.008632
1,-0.063823,-0.032015,-0.045244,-0.041999,-0.083964,-0.031247,-0.071268,-0.034561,-0.025514,-0.054092,...,-0.014648,-0.037029,-0.028536,-0.044369,-0.041867,-0.041334,-0.037026,-0.024260,-0.027980,-0.059786
2,-0.029444,-0.008522,0.015812,-0.003999,0.011968,-0.024144,0.002479,-0.010488,-0.004276,-0.005106,...,-0.014089,0.009162,-0.013693,-0.009630,-0.018619,-0.019764,-0.014604,-0.005417,-0.018452,-0.018390
3,-0.039021,-0.055268,-0.060448,-0.058737,-0.049801,-0.058354,-0.067091,-0.048430,-0.062268,-0.047026,...,-0.039420,-0.039512,-0.033167,-0.033066,-0.054215,-0.052686,-0.053998,-0.044555,-0.037007,-0.030520
4,0.045455,0.013603,0.028192,0.031247,0.031116,-0.015388,0.020125,0.004320,0.023154,-0.002884,...,0.031994,-0.012848,0.007665,0.005955,0.014265,0.004541,0.002496,-0.002457,-0.004309,0.008166
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1722,0.036969,0.018095,-0.003399,-0.010584,0.020175,0.064565,0.061905,0.053082,0.026207,0.055906,...,0.034756,0.043128,0.021623,0.027817,0.036353,0.037761,-0.022117,0.047906,0.027233,0.022715
1723,0.027280,-0.011080,-0.022809,-0.012979,-0.017596,0.057049,0.056897,0.023352,-0.006292,0.030830,...,0.026241,0.035098,0.023704,0.051456,0.029429,0.027584,0.004784,0.079743,0.040027,0.012807
1724,-0.005746,-0.041629,-0.023982,-0.026981,-0.004323,-0.007365,0.014685,-0.008915,-0.032651,-0.014849,...,-0.007610,-0.004272,-0.011019,-0.019899,-0.000957,-0.014075,-0.016517,-0.004843,-0.015740,-0.028945
1725,0.025919,0.001157,0.010012,0.005349,0.006464,0.022044,0.048221,0.039267,-0.011347,0.010883,...,0.018122,-0.003189,0.021419,0.032311,0.033128,0.034207,0.010462,0.022621,0.018573,0.012473


In [ ]:
K = 10 # number of factors

# 주간 수익률 데이터를 읽어온다.
rtn_df = pd.read_csv('49_Industry_Portfolios_Daily_Value_logreturn.csv', index_col=0)
rtn_R = np.array(rtn_df)

N_ROW = rtn_df.shape[0] # time 개수
N_COL = rtn_R.shape[1] # item (종목) 개수

# mean centering
E = rtn_R.mean(axis=0).reshape(1, N_COL)
cent_R = rtn_R - E

# User-item matrix
N = np.NaN
R = np.array(rtn_R)

print('\nExpected :')
print(E.round(4))

@nb.jit
# SGD로 행렬 F, B를 업데이트한다.
def update_matrix(R, F, B, a, r):
    for i in range(N_ROW):
        for j in range(N_COL):
            if np.isnan(R[i, j]) != True:  # nan이 아니면
                # error 항을 계산한다.
                eij = R[i, j] - np.dot(F[i, :], B[j, :])
                
                # update F, B
                F[i, :] += a * (eij * B[j, :] - r * F[i, :])
                B[j, :] += a * (eij * F[i, :] - r * B[j, :])

@nb.jit
# NaN이 포함된 행렬의 mean_squared_error를 계산한다. 행렬 x에는 NaN이 포함돼 있다. y에는 없다.
def mse_skip_nan(x, y):
    mse = 0.0
    cnt = 0
    for i in range(x.shape[0]):
        for j in range(x.shape[1]):
            if np.isnan(x[i, j]) != True:  # nan이 아니면
                mse += (x[i, j] - y[i, j]) ** 2
                cnt += 1
    return mse / cnt

# SGD로 행렬을 F, B로 분해한다.
def factorize_matrix(matR, k, max_iter=1000, alpha=0.0025, gamma=0.001, err_limit=1e-8):
    # F, B를 random 초기화한다.
    F = np.random.rand(N_ROW, k)  # factor matrix
    B = np.random.rand(N_COL, k)  # beta matrix.

    old_err = 9999   # error 초깃값
    err_hist = []    # error history
    for step in range(max_iter):
        # F, B를 업데이트한다.
        update_matrix(matR, F, B, alpha, gamma)

        # error를 계산하고 저장해 둔다.
        err = mse_skip_nan(matR, np.dot(F, B.T))
        err_hist.append(err)

        # early stopping
        if np.abs(old_err - err) < err_limit:
            break
        
        old_err = err

        if step % 10 == 0:
            print('{} : error={:.4f}'.format(step, err))

    if step >= max_iter - 1:
        print('max_iter={}번 동안 stop하지 못했습니다.'.format(max_iter))
        print('max_iter를 늘리거나 err_limit을 늘려야 합니다.')

    return F, B.T, err_hist

F, B, err = factorize_matrix(R, K)
ER = np.dot(F, B)   # estimated R

# error history를 관찰한다.
plt.plot(err, label='error')
plt.legend()
plt.title('error history')
plt.show()

print('\nR :')
print(np.round(R, 2))
print('\nEstimated R :')
print(np.round(ER, 2))   
print('\nF :')
print(np.round(F, 2))
print('\nB :')
print(np.round(B, 2))

In [ ]:
# F, B로 추정한 주가와 실제 주가 차트를 비교한다.
sym2idx = {k:v for v, k in enumerate(list(rtn_df.columns))}
idx2sym = {v:k for k, v in sym2idx.items()}

rtn_EFB = E + np.dot(F, B)  # 추정된 수익률
def calc_price(r):
    prc = [1.0]
    for i in r:
        prc.append(prc[-1] * np.exp(i))
    return prc

# 10행 5열의 plot chart를 그린다.
n_from = -200   # 최근 200 weeks (약 4년)의 주가 차트
for i in np.arange(0, N_COL, 5):
    fig = plt.figure(figsize=(12, 1.8))
    for j in np.arange(i, i+5):
        # 추정 주가와 실제 주가를 계산한다.
        pred_price = calc_price(rtn_EFB[:, j][n_from:])
        real_price = calc_price(rtn_R[:, j][n_from:])
        ax = fig.add_subplot(1, 5, j % 5 + 1)
        ax.plot(real_price, label='real')
        ax.plot(pred_price, label='pred')
        ax.legend()
        ax.set_title(idx2sym[j])
    plt.show() 